In [1]:
import numpy as np
import cplex
import random

In [2]:
class Subproblem_LP():
    def __init__(self,xval):
        self.n_dim = (n_stage - 1) * n_job * n_job
        self.n_constraints = 2 * (n_stage - 1) * n_job
        self.variable_names = ['x'+ str(i) for i in range(self.n_dim)]
        self.constraints_rows = self.set_constraints_byrows()
        self.opt_solution = xval
        self.obj = self.compute_obj(tval, lamd)
        self.rhs = [1] * self.n_constraints
        self.senses = 'E'* self.n_constraints
        self.Subproblem_LP_TU = cplex.Cplex()

        
    def set_job_to_position(self):
        job_to_position_onlyone = []
        for i in range((n_stage - 1) * n_job):
            job_to_position_onlyone.append([self.variable_names[i * n_job : (i + 1) * n_job], [1] * n_job])
        return job_to_position_onlyone
    
    def set_position_to_job(self):
        position_to_job_onlyone = []
        for j in range(n_stage - 1):
            for i in range(n_job):
                position_to_job_onlyone.append([self.variable_names[j * (n_job**2) + i: (j + 1) * (n_job**2): n_job], [1] * n_job])
        return position_to_job_onlyone
    
    def set_constraints_byrows(self):
        job_to_position_onlyone, position_to_job_onlyone = self.set_job_to_position(), self.set_position_to_job()
        return job_to_position_onlyone + position_to_job_onlyone
    
    def compute_obj(self, tval, lamd):
        x_coeff_f4 = self.compute_f4_x(lamd)
        x_coeff_F6 = self.compute_F6_x(tval, lamd)
        x_coeff = [0 for i in range((n_stage - 1) * n_job * n_job)]
        for i in range(n_job):
            for j in range(n_stage-1):
                for r in range(n_batch * pcast):
                    for k in range(n_machine_perstage):
                        for i1 in range(n_job):
                            t2 = i * n_job + j * n_job * n_job + k * n_batch * pcast + r
                            x_coeff[t2] = x_coeff_f4[t2] + x_coeff_F6[t2]
        return x_coeff
    
    def compute_F6_x(self, tval, lamd):
        x_coeff=[0 for i in range((n_stage - 1) * n_job * n_job)]
        for i in range(n_job):
            for j in range(n_stage-1):
                t1 = i + j * n_job
                for r in range(n_batch * pcast):
                    for k in range(n_machine_perstage):
                        t2 = i * n_job + j * n_job * n_job + k * n_batch * pcast + r
                        for i1 in range(n_job):
                            if r == n_batch * pcast - 1:
                                t3 = i1 * n_job + i * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r - 1
                                t4 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r - 1 
                                t5 = i1 + j * n_job
                                x_coeff[t2] += lamd[t3] * ((0.5 * ((self.opt_solution[t2] + self.opt_solution[t4])**2 - self.opt_solution[t2] - self.opt_solution[t4]) + tval[t1] -  tval[t5]) * (self.opt_solution[t2] + self.opt_solution[t4] - 0.5) - 0.5 * (2 * ProcessTime[t5] + 1) * (self.opt_solution[t4] - self.opt_solution[t2]))           
                            elif r == 0:
                                t3 = i * n_job + i1 * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r
                                t4 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r + 1
                                t5 = i1 + j * n_job
                                x_coeff[t2] += lamd[t3] * ((0.5 * ((self.opt_solution[t2] + self.opt_solution[t4])**2 - self.opt_solution[t2] - self.opt_solution[t4]) + tval[t5] -  tval[t1]) * (self.opt_solution[t2] + self.opt_solution[t4] - 0.5) + 0.5 * (2 * ProcessTime[t1] + 1) * (self.opt_solution[t2] - self.opt_solution[t4]))           
                            else:
                                t3 = i * n_job + i1 * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r
                                t4 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r + 1
                                t5 = i1 + j * n_job
                                t6 = i1 * n_job + i * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r - 1
                                t7 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r - 1
                                x_coeff[t2] += lamd[t3] * ((0.5 * ((self.opt_solution[t2] + self.opt_solution[t4])**2 - self.opt_solution[t2] - self.opt_solution[t4]) + tval[t5] -  tval[t1]) * (self.opt_solution[t2] + self.opt_solution[t4] - 0.5) + 0.5 * (2 * ProcessTime[t1] + 1) * (self.opt_solution[t2] - self.opt_solution[t4]))
                                x_coeff[t2] += lamd[t6] * ((0.5 * ((self.opt_solution[t2] + self.opt_solution[t7])**2 - self.opt_solution[t2] - self.opt_solution[t7]) + tval[t1] -  tval[t5]) * (self.opt_solution[t2] + self.opt_solution[t7] - 0.5) - 0.5 * (2 * ProcessTime[t5] + 1) * (self.opt_solution[t7] - self.opt_solution[t2]))           
        return x_coeff
    
    def compute_f4_x(self, lamd):
        x_coeff = [0 for i in range((n_stage - 1) * n_job * n_job)]
        for i in range(n_job):
            for j in range(n_stage - 1):
                t1 = i + j * n_job
                for r in range(n_batch * pcast):
                    for k in range(n_machine_perstage):
                        t2 = i * n_job + j * n_job * n_job + k * n_batch * pcast + r
                        for i1 in range(n_job):
                            if r == n_batch * pcast - 1:
                                t3 = i * n_job + i1 * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + n_batch * pcast - 1
                                x_coeff[t2] += 0.25 * (2 * ProcessTime[t1] + 1) * lamd[t3]
                            elif r == 0:
                                t3 = i * n_job + i1 * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast
                                x_coeff[t2] += 0.25 * (2 * ProcessTime[t1] + 1) * lamd[t3]
                            else:
                                t3 = i * n_job + i1 * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r
                                t4 = i1 * n_job + i * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r + 1
                                x_coeff[t2] += 0.25 * (2 * ProcessTime[t1] + 1) * (lamd[t3] + lamd[t4])
        return x_coeff

    def set_method(self, method):
        alg = self.Subproblem_LP_TU.parameters.lpmethod.values

        if method == "o":  
            self.Subproblem_LP_TU.parameters.lpmethod.set(alg.auto)
        elif method == "p":
            self.Subproblem_LP_TU.parameters.lpmethod.set(alg.primal)
        elif method == "d":
            self.Subproblem_LP_TU.parameters.lpmethod.set(alg.dual)
        elif method == "b":
            self.Subproblem_LP_TU.parameters.lpmethod.set(alg.barrier)
            self.Subproblem_LP_TU.parameters.barrier.crossover.set(self.Subproblem_LP_TU.parameters.parameters.barrier.crossover.values.none)
        elif method == "h":
            self.Subproblem_LP_TU.parameters.lpmethod.set(alg.barrier)
        elif method == "s":
            self.Subproblem_LP_TU.parameters.lpmethod.set(alg.sifting)
        elif method == "c":
            self.Subproblem_LP_TU.parameters.lpmethod.set(alg.concurrent)
        else:
            raise ValueError('method must be one of "o", "p", "d", "b", "h", "s" or "c"')
        
    def solve(self, method = 'o'):
        self.Subproblem_LP_TU.objective.set_sense(self.Subproblem_LP_TU.objective.sense.maximize)
        ub = [1] * self.n_dim
    
        self.Subproblem_LP_TU.variables.add(obj = self.obj, ub = ub, names = self.variable_names)    
    
        assert(self.n_constraints == len(self.constraints_rows))
        row_names = ['c'+ str(i) for i in range(self.n_constraints)]
        print("the dimension of x:", self.n_dim, "the number of constraints:", self.n_constraints)
        self.Subproblem_LP_TU.linear_constraints.add(lin_expr = self.constraints_rows, senses = self.senses, rhs = self.rhs, names = row_names)
        self.set_method(method)
        self.Subproblem_LP_TU.solve()
        self.get_solvable_status()
        self.get_optvalues()
        
    def write_model(self):
        self.Subproblem_LP_TU.write('Subproblem_LP.lp')
        
    def get_optvalues(self):
        for i, x in enumerate(self.Subproblem_LP_TU.solution.get_values()):
            if i < self.n_dim:
                self.opt_solution.append(x)
                
    def get_solvable_status(self):
        status = self.Subproblem_LP_TU.solution.get_status()
        if status == self.Subproblem_LP_TU.solution.status.unbounded:
            print("Model is unbounded")
        if status == self.Subproblem_LP_TU.solution.status.infeasible:
            print("Model is infeasible")
        if status == self.Subproblem_LP_TU.solution.status.infeasible_or_unbounded:
            print("Model is infeasible or unbounded")
            
    def output_LP_result(self):
        with open('Subproblem_LP_result.txt', 'w') as f:
            for i in range(n_job * n_stage):
                f.write(str(self.opt_solution[i * n_job:(i + 1) * n_job]))
            f.write('\n')

In [5]:
xval = [random.randint(0,1) for i in range((n_stage - 1) * n_job * n_job)]
subproblem_lp = Subproblem_LP(xval) 
job_to_position_onlyone = subproblem_lp.set_job_to_position()
print(len(job_to_position_onlyone))
print(job_to_position_onlyone)

[1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1]
[1, 0, 1, 0, 0, 0]
[0, 1, 0, 0, 0, 1]
[1, 0, 0, 0, 1, 1]
[0, 0, 0, 1, 1, 1]
[0, 0, 1, 1, 1, 0]
[0, 1, 1, 1, 0, 1]
[1, 1, 1, 0, 1, 1]
[1, 1, 0, 1, 1, 0]
[1, 0, 1, 1, 0, 0]
[0, 1, 1, 0, 0, 0]
[1, 1, 0, 0, 0, 1]
[1, 0, 0, 0, 1, 1]
[0, 0, 0, 1, 1, 1]
[0, 0, 1, 1, 1, 1]
[0, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 0]
[1, 1, 1, 1, 0, 1]
[1, 1, 1, 0, 1, 1]
[1, 1, 0, 1, 1, 0]
[1, 0, 1, 1, 0, 0]
[0, 1, 1, 0, 0, 0]
[1, 1, 0, 0, 0, 1]
[1, 0, 0, 0, 1, 0]
[0, 0, 0, 1, 0, 0]
[0, 0, 1, 0, 0, 1]
[0, 1, 0, 0, 1, 1]
[1, 0, 0, 1, 1, 1]
[0, 0, 1, 1, 1, 1]
[0, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 0]
[1, 1, 1, 1, 0, 0]
[1, 1, 1, 0, 0, 0]
[1, 1, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 1, 1]
[0, 0, 0, 1, 1, 1]
[0, 0, 1, 1, 1, 0]
[0, 1, 1, 1, 0, 1]
[1, 1, 1, 0, 1, 1]
[1, 1, 0, 1, 1, 1]
[1, 0, 1, 1, 1, 0]
[0, 1, 1, 1, 0, 0]
[1, 1, 1, 0, 0, 0]
[1, 1, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 1, 1]
[0, 0, 0, 1, 

In [6]:
xval = [random.randint(0,1) for i in range((n_stage - 1) * n_job * n_job)]
subproblem_lp = Subproblem_LP(xval) 
position_to_job_onlyone = subproblem_lp.set_position_to_job()
print(len(position_to_job_onlyone))
print(position_to_job_onlyone)

[1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0]
[0, 0, 0, 1, 0, 0]
[0, 0, 1, 0, 0, 0]
[0, 1, 0, 0, 0, 1]
[1, 0, 0, 0, 1, 1]
[0, 0, 0, 1, 1, 1]
[0, 0, 1, 1, 1, 1]
[0, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 0]
[1, 1, 1, 1, 0, 0]
[1, 1, 1, 0, 0, 1]
[1, 1, 0, 0, 1, 0]
[1, 0, 0, 1, 0, 0]
[0, 0, 1, 0, 0, 1]
[0, 1, 0, 0, 1, 0]
[1, 0, 0, 1, 0, 0]
[0, 0, 1, 0, 0, 0]
[0, 1, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 1, 1]
[0, 0, 0, 1, 1, 1]
[0, 0, 1, 1, 1, 0]
[0, 1, 1, 1, 0, 0]
[1, 1, 1, 0, 0, 0]
[1, 1, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 1, 1]
[0, 0, 0, 1, 1, 0]
[0, 0, 1, 1, 0, 1]
[0, 1, 1, 0, 1, 0]
[1, 1, 0, 1, 0, 1]
[1, 0, 1, 0, 1, 1]
[0, 1, 0, 1, 1, 0]
[1, 0, 1, 1, 0, 0]
[0, 1, 1, 0, 0, 1]
[1, 1, 0, 0, 1, 1]
[1, 0, 0, 1, 1, 0]
[0, 0, 1, 1, 0, 1]
[0, 1, 1, 0, 1, 0]
[1, 1, 0, 1, 0, 0]
[1, 0, 1, 0, 0, 0]
[0, 1, 0, 0, 0, 1]
[1, 0, 0, 0, 1, 0]
[0, 0, 0, 1, 0, 1]
[0, 0, 1, 0, 1, 1]
[0, 1, 0, 1, 1, 1]
[1, 0, 1, 1, 1, 1]
[0, 1, 1, 1, 1, 0]
[1, 1, 1, 1, 

In [4]:
n_stage, pcast, n_batch, n_machine_perstage = 3, 1, 3, 2
stage_name = ['steelmaking', 'refining', 'casting']
n_machine = dict(zip(stage_name, [n_machine_perstage for i in range(n_stage)]))
n_job = pcast * n_batch * n_machine['steelmaking']
assert(len(stage_name) == n_stage)

print("Number of stage:", n_stage)
print("Cast number of per machine:", pcast)
print("Number of job per cast:", n_batch)
print('Number of machine:', n_machine)
print('Number of job:', n_job)

n_position = dict(zip(stage_name[0:n_stage - 1], [n_job / n_machine['steelmaking'] ] * (n_stage - 1)))
print('n_position = ', n_position)
assert(len(n_machine) == n_stage)
assert(len(n_position) == n_stage - 1)

tval = [random.randint(0,1) for i in range((n_stage - 1) * n_job)]
lamd = [1 for i in range((n_stage - 1) * n_job * n_job * n_job)]
ProcessTime = [random.randint(0,1) for i in range((n_stage - 1) * n_job)]
xval = [random.randint(0,1) for i in range((n_stage - 1) * n_job * n_job)]
subproblem_lp = Subproblem_LP(xval) 

lamd = [1 for i in range((n_stage - 1) * n_job * n_job * n_job)]
ProcessTime = [random.randint(30,36) for i in range((n_stage - 1) * n_job)]
print(subproblem_lp.compute_f4_x(lamd))

Number of stage: 3
Cast number of per machine: 1
Number of job per cast: 3
Number of machine: {'steelmaking': 2, 'refining': 2, 'casting': 2}
Number of job: 6
n_position =  {'steelmaking': 3.0, 'refining': 3.0}
[0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0]
[0, 1, 1, 0, 0, 1]
[1, 1, 0, 0, 1, 1]
[1, 0, 0, 1, 1, 1]
[0, 0, 1, 1, 1, 1]
[0, 1, 1, 1, 1, 0]
[1, 1, 1, 1, 0, 0]
[1, 1, 1, 0, 0, 1]
[1, 1, 0, 0, 1, 0]
[1, 0, 0, 1, 0, 0]
[0, 0, 1, 0, 0, 0]
[0, 1, 0, 0, 0, 1]
[1, 0, 0, 0, 1, 1]
[0, 0, 0, 1, 1, 0]
[0, 0, 1, 1, 0, 0]
[0, 1, 1, 0, 0, 0]
[1, 1, 0, 0, 0, 1]
[1, 0, 0, 0, 1, 0]
[0, 0, 0, 1, 0, 1]
[0, 0, 1, 0, 1, 1]
[0, 1, 0, 1, 1, 1]
[1, 0, 1, 1, 1, 1]
[0, 1, 1, 1, 1, 0]
[1, 1, 1, 1, 0, 1]
[1, 1, 1, 0, 1, 0]
[1, 1, 0, 1, 0, 0]
[1, 0, 1, 0, 0, 0]
[0, 1, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 1, 0]
[0, 0, 0, 1, 0, 0]
[0, 0, 1, 0, 0, 0]
[0, 1, 0, 0, 0, 1]
[1, 0, 0, 0, 1, 1]
[0, 0, 0, 1, 1, 1]
[0, 0, 1, 1, 1, 1]
[0, 1, 1, 1, 1, 0]
[1, 1, 1, 1, 0, 0]
[1, 1, 1, 0, 0, 1]
[1, 1, 0, 0, 1, 1]
[1, 0, 0, 1

In [7]:
class Subproblem_QP():
    def __init__(self, tval):
        self.n_dim = n_job * n_stage + 2 * pcast * n_machine['steelmaking']
        self.variable_names = ['t' + str(i)  for i in range(self.n_dim)]
        self.Subproblem_QP = cplex.Cplex()
        self.duedate = self.compute_duedate()
        self.constraints_rows, self.rhs, self.n_per_constraints = self.set_constraints_byrows()
        self.senses = self.set_senses()
        #self.obj_linear, self.qmat = self.compute_obj_linear(), self.compute_qmat()
        self.opt_solution = tval
        self.n_constraints = len(self.constraints_rows)
        
    def set_preceding_transfer_constraints(self):
        preceding_transfer_constraints = []
        for i in range(n_job * (n_stage - 1)):
            preceding_transfer_constraints.append([[self.variable_names[i], self.variable_names[i + n_job]], [1, -1]])

        rhs_transittime, rhs_processtime = [], processtime[0: n_job * (n_stage - 1)]
        for key in transittime:
            rhs_transittime += [transittime[key]] * n_job
        rhs = [ - processtime[i] - rhs_transittime[i] for i in range(n_job * (n_stage - 1))]
        return preceding_transfer_constraints, rhs, rhs_processtime, rhs_transittime

    def set_process_continuous_constraints(self):
        process_continuous, adjacent_casts = [], []
        rhs_continuous_processtime, rhs_adjacent_casts = [], []
        assert(n_job * (n_stage - 1) % n_batch == 0)
        for i in range(n_job * (n_stage - 1), n_job * n_stage - 1):
            if (i + 1) % n_batch != 0:
                process_continuous.append([[self.variable_names[i], self.variable_names[i + 1]], [-1, 1]])
                rhs_continuous_processtime.append(processtime[i])
            else:
                if (i + 1) % (n_batch * pcast) != 0:
                    adjacent_casts.append([[self.variable_names[i], self.variable_names[i + 1]], [-1, 1]])
                    rhs_adjacent_casts.append(- processtime[i] - su)
        return process_continuous, adjacent_casts, rhs_continuous_processtime, rhs_adjacent_casts 

    def set_additional_constraints(self):
        additional_constraints = []
        index = n_job * n_stage
        index_start = pcast * n_machine['casting']
        assert(n_job * (n_stage - 1) % n_batch == 0 and (index % n_batch == 0))
        batch = 0
        for i in range(n_job * (n_stage - 1), n_job * n_stage - 1):
            if i % n_batch == 0:
                additional_constraints.append([[self.variable_names[i], self.variable_names[index + batch], self.variable_names[index + batch + index_start]], [1, -1, 1]])
                batch += 1
        return additional_constraints
    
    def set_constraints_byrows(self):
        preceding_transfer_constraints, rhs_preceding_transfer, _, _ = self.set_preceding_transfer_constraints()
        process_continuous_constraints, adjacent_casts_constraints, rhs_continuous_processtime, rhs_adjacent_casts = self.set_process_continuous_constraints()
        additional_constraints = self.set_additional_constraints()
    
        rows = preceding_transfer_constraints + process_continuous_constraints + adjacent_casts_constraints + additional_constraints
        
        n_per_constraints = [len(preceding_transfer_constraints), len(process_continuous_constraints), len(adjacent_casts_constraints), len(additional_constraints)]
        
        rhs = rhs_preceding_transfer + rhs_continuous_processtime + rhs_adjacent_casts + self.duedate
        return rows, rhs, n_per_constraints
    
    def set_senses(self):
        s, senses = 'LELE', ''
        for i in range(len(self.n_per_constraints)):
            senses += s[i] * self.n_per_constraints[i]
        return senses
    
    def compute_duedate(self):
        duedate = [0 for i in range(pcast * n_machine['casting'])]
        duedate_start = [0 for i in range(n_machine['casting'])]
        n_cast = pcast * n_machine['casting']
        for i in range(n_machine['casting']):
            index = 0
            for key in transittime:
                duedate_start[i] += processtime[i * n_cast + index * n_job] + transittime[key]
                index += 1
        print(duedate_start)
    
        for i in range(n_machine['casting']):
            duedate[i * pcast] = duedate_start[i]
            index_start = n_job * (n_stage - 1) + i * n_cast
            for j in range(1, pcast):
                duedate[i * pcast + j] = duedate[i * pcast + j - 1] + sum(processtime[index_start: index_start + n_batch]) + su
                index_start = index_start + n_batch
        return duedate
    
    def compute_obj_wait_time(self):
        obj_wait_time = np.zeros((n_stage, n_job))
        for i in range(n_stage - 1):
            for j in range(n_job):
                obj_wait_time[i + 1][j], obj_wait_time[i][j] = obj_wait_time[i + 1][j] + coeff[i], obj_wait_time[i][j] - coeff[i]
        obj_wait_time = obj_wait_time.reshape(1, n_stage * n_job)
        return np.hstack((obj_wait_time, np.zeros((1, self.n_dim - n_stage * n_job)))).tolist()[0]

    def compute_obj_casting(self):
        obj_casting = np.hstack((d_coeff['d1'] *  np.ones((1, pcast * n_machine['casting'])), d_coeff['d2'] * np.ones((1, pcast * n_machine['casting']))))
        obj_casting = np.hstack((np.zeros((1, n_job * n_stage)), obj_casting))
        return obj_casting.tolist()[0]
    
    def compute_obj_linear(self, xval, lamd):
        obj_linear_wait_time = self.compute_obj_wait_time()
        obj_linear_casting = self.compute_obj_casting()
        obj_linear_F6_t = self.compute_F6_t(xval, lamd)
        obj_linear = [obj_linear_wait_time[i] + obj_linear_casting[i] + obj_linear_F6_t[i] for i in range(self.n_dim)]
        return obj_linear
    
    def compute_F6_t(self, xval, lamd):
        for i in range((n_stage - 1) * n_job * n_job - n_job):
            print(xval[i:i + n_job])
            i += n_job
        t_coeff = [0 for i in range((n_stage - 1) * n_job)]
        for i in range(n_job):
            for j in range(n_stage - 1):
                t1 = i + j * n_job
                for r in range(n_batch * pcast):
                    for k in range(n_machine_perstage):
                        t2 = i * n_job + j * n_job * n_job + k * n_batch * pcast + r
                        for i1 in range(n_job):
                            if r == n_batch * pcast - 1:
                                t3 = i1 * n_job + i * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r - 1
                                t4 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r - 1 
                                t5 = i1 + j * n_job
                                t_coeff[t1] += lamd[t3] * (0.5 * ((xval[t2] + xval[t4])**2 - xval[t2] - xval[t4]) + self.opt_solution[t1] -  self.opt_solution[t5])          
                            elif r == 0:
                                t3 = i * n_job + i1 * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r
                                t4 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r + 1
                                t5 = i1 + j * n_job
                                t_coeff[t1] -= lamd[t3] * (0.5 * ((xval[t2] + xval[t4])**2 - xval[t2] - xval[t4]) + self.opt_solution[t5] -  self.opt_solution[t1])
                            else:
                                t3 = i * n_job + i1 * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r
                                t4 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r + 1
                                t5 = i1 + j * n_job
                                t6 = i1 * n_job + i * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r - 1
                                t7 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r - 1
                                t_coeff[t1] -= lamd[t3] * (0.5 * ((xval[t2] + xval[t4])**2 - xval[t2] - xval[t4]) + self.opt_solution[t5] -  self.opt_solution[t1]) 
                                t_coeff[t1] += lamd[t6] * (0.5 * ((xval[t2] + xval[t7])**2 - xval[t2] - xval[t7]) + self.opt_solution[t1] -  self.opt_solution[t5])            
        return t_coeff + [0] * (self.n_dim - len(t_coeff))
    
    def compute_qmat(self, lamd):
        quad_t_coeff = np.zeros((self.n_dim, self.n_dim))
        for j in range(n_stage-1):
            for r in range(n_batch * pcast-1):
                for k in range(n_machine_perstage):
                    for i in range(n_job):
                        for i1 in range(n_job):
                            if i1 != i:
                                t1 = i * n_job + i1 * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r
                                t2 = i1 * n_job + i * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r
                                quad_t_coeff[i + j * n_job, i + j * n_job] += 0.5 * (lamd[t1] + lamd[t2])
                    for i in range(n_job-1):
                        for i1 in range(i+1,n_job):
                            t1 = i * n_job + i1 * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r
                            t2 = i1 * n_job + i * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r
                            quad_t_coeff[i + j * n_job, i1 + j * n_job] -= 0.5 * (lamd[t1] + lamd[t2])
                            quad_t_coeff[i1 + j * n_job, i + j * n_job] -= 0.5 * (lamd[t1] + lamd[t2])                            
        quad_t_coeff = quad_t_coeff.tolist()
        return [[self.variable_names, quad_t_coeff[i]] for i in range(len(quad_t_coeff))]
    
    def solve(self):
        self.Subproblem_QP.objective.set_sense(self.Subproblem_QP.objective.sense.maximize)
        
        ub   = [cplex.infinity] * self.n_dim
        self.Subproblem_QP.variables.add(obj = self.obj_linear, ub = ub, names = self.variable_names)
        self.Subproblem_QP.objective.set_quadratic(self.qmat)
    
        rownames = ['c' + str(i)  for i in range(self.n_constraints)]

        self.Subproblem_QP.linear_constraints.add(lin_expr = self.constraints_rows, senses = self.senses, rhs = self.rhs, names = rownames)
        self.Subproblem_QP.solve()
        self.get_solvable_status()
        self.get_optvalues()

    def write_model(self):
        self.Subproblem_QP.write('Subproblem_QP.lp')
        
    def get_optvalues(self):
        for i, x in enumerate(self.Subproblem_QP.solution.get_values()):
            if i < self.n_dim:
                self.opt_solution.append(x)
                
    def get_solvable_status(self):
        status = self.Subproblem_QP.solution.get_status()
        if status == self.Subproblem_QP.solution.status.unbounded:
            print("Model is unbounded")
        if status == self.Subproblem_QP.solution.status.infeasible:
            print("Model is infeasible")
        if status == self.Subproblem_QP.solution.status.infeasible_or_unbounded:
            print("Model is infeasible or unbounded")
            
    def output_QP_result(self):
        with open('Subproblem_QP_result.txt', 'w') as f:
            for i in range(n_job * n_stage):
                f.write(str(self.opt_solution[i * n_job:(i + 1) * n_job]))
            f.write('\n')

In [10]:
subproblem_qp= Subproblem_QP(tval)
print(subproblem_qp.compute_obj_wait_time())
print(subproblem_qp.compute_obj_casting())

[81.40368721122313, 81.98582891234133]
[-10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -20.0, -20.0, -20.0, -20.0, -20.0, -20.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 10.0, 110.0, 110.0]


In [11]:
subproblem_qp= Subproblem_QP(tval)
preceding_transfer_constraints, rhs_preceding_transfer, rhs_processtime, rhs_transittime = subproblem_qp.set_preceding_transfer_constraints()
print(preceding_transfer_constraints)
print(rhs_processtime)
print(len(rhs_processtime))
print(rhs_transittime)
print(len(rhs_transittime))
print(rhs_preceding_transfer)
print(len(rhs_preceding_transfer))

[81.40368721122313, 81.98582891234133]
[[['t0', 't6'], [1, -1]], [['t1', 't7'], [1, -1]], [['t2', 't8'], [1, -1]], [['t3', 't9'], [1, -1]], [['t4', 't10'], [1, -1]], [['t5', 't11'], [1, -1]], [['t6', 't12'], [1, -1]], [['t7', 't13'], [1, -1]], [['t8', 't14'], [1, -1]], [['t9', 't15'], [1, -1]], [['t10', 't16'], [1, -1]], [['t11', 't17'], [1, -1]]]
[37.23059907628989, 45.117538698708536, 36.33136016443329, 44.79079961303195, 43.897300819188736, 37.18238057721895, 36.14909498818659, 37.22919611879861, 37.63047560116139, 44.137918290140576, 45.78027130377454, 43.198178175072805]
12
[3.7548019554214433, 3.7548019554214433, 3.7548019554214433, 3.7548019554214433, 3.7548019554214433, 3.7548019554214433, 4.269191191325202, 4.269191191325202, 4.269191191325202, 4.269191191325202, 4.269191191325202, 4.269191191325202]
12
[-40.98540103171133, -48.87234065412998, -40.086162119854734, -48.54560156845339, -47.65210277461018, -40.93718253264039, -40.418286179511796, -41.49838731012382, -41.899666792

In [12]:
subproblem_qp = Subproblem_QP(tval)
process_continuous_constraints, adjacent_casts_constraints, rhs_continuous_processtime, rhs_adjacent_casts =  subproblem_qp.set_process_continuous_constraints()
print(process_continuous_constraints)
print(rhs_continuous_processtime)
print(adjacent_casts_constraints)
print(rhs_adjacent_casts)

[81.40368721122313, 81.98582891234133]
[[['t12', 't13'], [-1, 1]], [['t13', 't14'], [-1, 1]], [['t15', 't16'], [-1, 1]], [['t16', 't17'], [-1, 1]]]
[47.256725727027685, 36.4281881572717, 37.46746543606981, 48.98286996456444]
[]
[]


In [13]:
subproblem_qp = Subproblem_QP(tval)
additional_constraints = subproblem_qp.set_additional_constraints()
print(additional_constraints)

[81.40368721122313, 81.98582891234133]
[[['t12', 't18', 't20'], [1, -1, 1]], [['t15', 't19', 't21'], [1, -1, 1]]]


In [14]:
subproblem_qp = Subproblem_QP(tval)
duedate = subproblem_qp.compute_duedate()
print(duedate)
print(len(duedate))
print(processtime)

[81.40368721122313, 81.98582891234133]
[81.40368721122313, 81.98582891234133]
[81.40368721122313, 81.98582891234133]
2
[37.23059907628989, 45.117538698708536, 36.33136016443329, 44.79079961303195, 43.897300819188736, 37.18238057721895, 36.14909498818659, 37.22919611879861, 37.63047560116139, 44.137918290140576, 45.78027130377454, 43.198178175072805, 47.256725727027685, 36.4281881572717, 40.59105916117858, 37.46746543606981, 48.98286996456444, 39.71151752968862]


In [16]:
subproblem_qp = Subproblem_QP(tval)
obj_wait_time = subproblem_qp.compute_obj_wait_time()
print(obj_wait_time)

[81.40368721122313, 81.98582891234133]
[-10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -20.0, -20.0, -20.0, -20.0, -20.0, -20.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 0.0, 0.0, 0.0, 0.0]


In [17]:
subproblem_qp = Subproblem_QP(tval)
obj_casting = subproblem_qp.compute_obj_casting()
print(obj_casting)
print(len(obj_casting))

[81.40368721122313, 81.98582891234133]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 10.0, 110.0, 110.0]
22


In [18]:
tval = [random.randint(0,1) for i in range((n_stage - 1) * n_job)]
subproblem_qp = Subproblem_QP(tval)
print('opt_solution = ', subproblem_qp.opt_solution)
lamd = [1 for i in range((n_stage - 1) * n_job * n_job * n_job)]
xval = [random.randint(0,1) for i in range((n_stage - 1) * n_job * n_job)]
t_coeff = subproblem_qp.compute_F6_t(xval, lamd)
print(len(t_coeff))
print(t_coeff)

[81.40368721122313, 81.98582891234133]
opt_solution =  [1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1]
[1, 1, 1, 1, 0, 0]
[1, 1, 1, 0, 0, 0]
[1, 1, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 1, 0]
[0, 0, 0, 1, 0, 0]
[0, 0, 1, 0, 0, 0]
[0, 1, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 1, 1]
[0, 0, 0, 1, 1, 0]
[0, 0, 1, 1, 0, 1]
[0, 1, 1, 0, 1, 1]
[1, 1, 0, 1, 1, 1]
[1, 0, 1, 1, 1, 1]
[0, 1, 1, 1, 1, 0]
[1, 1, 1, 1, 0, 1]
[1, 1, 1, 0, 1, 1]
[1, 1, 0, 1, 1, 0]
[1, 0, 1, 1, 0, 0]
[0, 1, 1, 0, 0, 1]
[1, 1, 0, 0, 1, 0]
[1, 0, 0, 1, 0, 1]
[0, 0, 1, 0, 1, 0]
[0, 1, 0, 1, 0, 0]
[1, 0, 1, 0, 0, 0]
[0, 1, 0, 0, 0, 1]
[1, 0, 0, 0, 1, 0]
[0, 0, 0, 1, 0, 1]
[0, 0, 1, 0, 1, 1]
[0, 1, 0, 1, 1, 1]
[1, 0, 1, 1, 1, 0]
[0, 1, 1, 1, 0, 0]
[1, 1, 1, 0, 0, 1]
[1, 1, 0, 0, 1, 1]
[1, 0, 0, 1, 1, 1]
[0, 0, 1, 1, 1, 1]
[0, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 0]
[1, 1, 1, 1, 0, 0]
[1, 1, 1, 0, 0, 1]
[1, 1, 0, 0, 1, 1]
[1, 0, 0, 1, 1, 0]
[0, 0, 1, 1, 0, 1]
[0, 1, 1, 0, 1,

In [19]:
lamd = [1 for i in range((n_stage - 1) * n_job * n_job * n_job)]
qmat = subproblem_qp.compute_qmat(lamd)
print(qmat)
print(len(qmat))

[[['t0', 't1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10', 't11', 't12', 't13', 't14', 't15', 't16', 't17', 't18', 't19', 't20', 't21'], [20.0, -4.0, -4.0, -4.0, -4.0, -4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], [['t0', 't1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10', 't11', 't12', 't13', 't14', 't15', 't16', 't17', 't18', 't19', 't20', 't21'], [-4.0, 20.0, -4.0, -4.0, -4.0, -4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], [['t0', 't1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10', 't11', 't12', 't13', 't14', 't15', 't16', 't17', 't18', 't19', 't20', 't21'], [-4.0, -4.0, 20.0, -4.0, -4.0, -4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], [['t0', 't1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10', 't11', 't12', 't13', 't14', 't15', 't16', 't17', 't18', 't19', 't20', 't21'], [-4.0, -4.0, -4.0, 20.0, -4.0, -4.0, 0.0, 0.0, 0.0, 0.

In [ ]:
tval = [random.randint(0,1) for i in range((n_stage - 1) * n_job)]
subproblem_qp = Subproblem_QP(tval)
print('opt_solution = ', subproblem_qp.opt_solution)
lamd = [1 for i in range((n_stage - 1) * n_job * n_job * n_job)]
xval = [random.randint(0,1) for i in range((n_stage - 1) * n_job * n_job)]
print(subproblem_qp.compute_obj_linear(xval, lamd))

In [9]:
n_stage, pcast, n_batch, n_machine_perstage = 3, 1, 3, 2
stage_name = ['steelmaking', 'refining', 'casting']
n_machine = dict(zip(stage_name, [n_machine_perstage for i in range(n_stage)]))
n_job = pcast * n_batch * n_machine['steelmaking']

processtime = [14 * random.random() + 36 for i in range(n_job * n_stage)]
transittime = dict(zip([stage_name[i] + 'to' + stage_name[i+1] for i in range(n_stage - 1)], [3 * random.random() + 3 for i in range(n_stage - 1)]))
su = 80
duedate = [0 for i in range(pcast * n_machine['casting'])]

penalty_coeff = 10
d_coeff = {'d1': 10, 'd2': 110}
coeff = [10 + 20 * i for i in range(n_stage - 1)]

assert(len(stage_name) == n_stage)
print("Number of stage:", n_stage)
print("Cast number of per machine:", pcast)
print("Number of machine per stage:", n_batch)
print('Number of machine:', n_machine)
print('Number of job:', n_job)
print('transfer time:', transittime)

n_position = dict(zip(stage_name[0:n_stage - 1], [n_job / n_machine['steelmaking'] ] * (n_stage - 1)))
print('n_position = ', n_position)
assert(len(n_machine) == n_stage)
assert(len(n_position) == n_stage - 1)

Number of stage: 3
Cast number of per machine: 1
Number of machine per stage: 3
Number of machine: {'steelmaking': 2, 'refining': 2, 'casting': 2}
Number of job: 6
transfer time: {'steelmakingtorefining': 3.7548019554214433, 'refiningtocasting': 4.269191191325202}
n_position =  {'steelmaking': 3.0, 'refining': 3.0}


In [ ]:
n_stage, pcast, n_batch, n_machine_perstage = 3, 5, 3, 3
stage_name = ['steelmaking', 'refining', 'casting']
n_machine = dict(zip(stage_name, [n_machine_perstage for i in range(n_stage)]))
n_job = pcast * n_batch * n_machine['steelmaking']

processtime = [14 * random.random() + 36 for i in range(n_job * n_stage)]
transittime = dict(zip([stage_name[i] + 'to' + stage_name[i+1] for i in range(n_stage - 1)], [3 * random.random() + 3 for i in range(n_stage - 1)]))
su = 80
duedate = [0 for i in range(pcast * n_machine['casting'])]

penalty_coeff = 10
d_coeff = {'d1': 10, 'd2': 110}
coeff = [10 + 20 * i for i in range(n_stage - 1)]

assert(len(stage_name) == n_stage)
print("Number of stage:", n_stage)
print("Cast number of per machine:", pcast)
print("Number of machine per stage:", n_batch)
print('Number of machine:', n_machine)
print('Number of job:', n_job)
print('transfer time:', transittime)

n_position = dict(zip(stage_name[0:n_stage - 1], [n_job / n_machine['steelmaking'] ] * (n_stage - 1)))
print('n_position = ', n_position)
assert(len(n_machine) == n_stage)
assert(len(n_position) == n_stage - 1)

In [ ]:
m, r = 2, 0.01
lambd = np.zeros((n_job, n_job, n_job, n_stage))
maxiteration_times = 10
c0 = 1
g0 = np.ones((n_job, n_job, n_job, n_stage))
gm = compute_subgradients()
for iteration_times in range(1, maxiteration_times):
    stepsize = compute_stepsize(iteration_times, gm)
    print(stepsize)

In [ ]:
def compute_subgradients():
    return np.ones((lambd.shape))

def compute_stepsize(i, gm):
    p = 1 - 1/(i ** r)
    alpha = 1
    for j in range(1, i + 1):
        alpha = alpha * (1 - 1/(m * j ** p))
    if np.linalg.norm(gm) != 0:
        c = alpha * c0 * np.linalg.norm(g0) / np.linalg.norm(gm)
    return c